In [1]:
from darknet import Darknet
import dataset
import torch
from torch.autograd import Variable
from torchvision import datasets, transforms
from utils import *
import os

datacfg = 'cfg/voc.data'
cfgfile = 'cfg/yolo-voc.cfg'
weightfile = 'weights/000510.weights'
outfile = 'comp4_det_test_'


options = read_data_cfg(datacfg)
valid_images = options['valid']
name_list = options['names']
prefix = 'results'
names = load_class_names(name_list)


In [2]:
options

{'gpus': '0,1',
 'num_workers': '10',
 'train': 'data/voc_train.txt',
 'valid': 'data/voc_train.txt',
 'names': 'data/an.names',
 'backup': 'backup'}

In [3]:

with open(valid_images) as fp:
    tmp_files = fp.readlines()
    valid_files = [item.rstrip() for item in tmp_files]

m = Darknet(cfgfile)
m.print_network()
m.load_weights(weightfile)
m.cuda()
m.eval()

valid_dataset = dataset.listDataset(valid_images, shape=(m.width, m.height),
                   shuffle=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                   ]))
valid_batchsize = 16
assert(valid_batchsize > 1)

kwargs = {'num_workers': 4, 'pin_memory': True}
valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=valid_batchsize, shuffle=False, **kwargs) 

fps = [0]*m.num_classes
if not os.path.exists('results'):
    os.mkdir('results')
for i in range(m.num_classes):
    buf = '%s/%s%s.txt' % (prefix, outfile, names[i])
    fps[i] = open(buf, 'w')

lineId = -1

conf_thresh = 0
nms_thresh = 0.30
for batch_idx, (data, target) in enumerate(valid_loader):
    data = data.cuda()
    data = Variable(data, volatile = True)
    output = m(data).data
    batch_boxes = get_region_boxes(output, conf_thresh, m.num_classes, m.anchors, m.num_anchors, 0, 1)
    for i in range(output.size(0)):
        lineId = lineId + 1
        fileId = os.path.basename(valid_files[lineId]).split('.')[0]
        width, height = get_image_size(valid_files[lineId])
        print(valid_files[lineId])
        boxes = batch_boxes[i]
        boxes = nms(boxes, nms_thresh)
        for box in boxes:
            x1 = (box[0] - box[2]/2.0) * width
            y1 = (box[1] - box[3]/2.0) * height
            x2 = (box[0] + box[2]/2.0) * width
            y2 = (box[1] + box[3]/2.0) * height

            det_conf = box[4]
            for j in range((len(box)-5)/2):
                cls_conf = box[5+2*j]
                cls_id = box[6+2*j]
                prob =det_conf * cls_conf
                fps[cls_id].write('%s %f %f %f %f %f\n' % (fileId, prob, x1, y1, x2, y2))

for i in range(m.num_classes):
    fps[i].close()



layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32
    1 max          2 x 2 / 2   416 x 416 x  32   ->   208 x 208 x  32
    2 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64
    3 max          2 x 2 / 2   208 x 208 x  64   ->   104 x 104 x  64
    4 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
    5 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64
    6 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
    7 max          2 x 2 / 2   104 x 104 x 128   ->    52 x  52 x 128
    8 conv    256  3 x 3 / 1    52 x  52 x 128   ->    52 x  52 x 256
    9 conv    128  1 x 1 / 1    52 x  52 x 256   ->    52 x  52 x 128
   10 conv    256  3 x 3 / 1    52 x  52 x 128   ->    52 x  52 x 256
   11 max          2 x 2 / 2    52 x  52 x 256   ->    26 x  26 x 256
   12 conv    512  3 x 3 / 1    26 x  26 x 256   ->    26 x  26 x 512
   13 conv    256  1 x 

RuntimeError: shape '[125, 1024, 1, 1]' is invalid for input of size 46000

In [1]:
m

NameError: name 'm' is not defined